WARNING: this example is not yet fully functional

In [1]:
from line_solver import *
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
model = Network('Model')
# Block 1: nodes
clientDelay = Delay(model, 'Client')
cacheNode = Cache(model, 'Cache', 1000, 50, ReplacementStrategy.LRU)
cacheDelay = Delay(model, 'CacheDelay')

In [3]:
# Block 2: classes
clientClass = ClosedClass(model, 'ClientClass', 1, clientDelay, 0)
hitClass = ClosedClass(model, 'HitClass', 0, clientDelay, 0)
missClass = ClosedClass(model, 'MissClass', 0, clientDelay, 0)

clientDelay.setService(clientClass, Immediate())
cacheDelay.setService(hitClass, Exp.fitMean(0.2))
cacheDelay.setService(missClass, Exp.fitMean(1.0))

cacheNode.setRead(clientClass, Zipf(1.4, 1000))
cacheNode.setHitClass(clientClass, hitClass)
cacheNode.setMissClass(clientClass, missClass)

In [4]:
# Block 3: topology
P = model.initRoutingMatrix()
# routing from client to cache
P.set(clientClass, clientClass, clientDelay, cacheNode, 1.0)
# routing out of the cache
P.set(hitClass, hitClass, cacheNode, cacheDelay, 1.0)
P.set(missClass, missClass, cacheNode, cacheDelay, 1.0)
# return to the client
P.set(hitClass, clientClass, cacheDelay, clientDelay, 1.0)
P.set(missClass, clientClass, cacheDelay, clientDelay, 1.0)
# routing from cacheNode
model.link(P)

In [5]:
# SSA not working on this case in Java but MVA works fine
# Block 4: solution
mvaAvgTable = SolverMVA(model,'verbose',True).getAvgTable()

      Station     JobClass        QLen        Util       RespT      ResidT  \
0      Client  ClientClass  1.5285e-05  2.6108e-08  5.8546e-06  5.8546e-06   
4  CacheDelay     HitClass  4.0268e-01  4.0269e-01  2.0000e-01  1.5424e-01   
5  CacheDelay    MissClass  5.9730e-01  5.9731e-01  9.9998e-01  2.2878e-01   

   ArvR    Tput  
0   0.0  2.6108  
4   0.0  2.0134  
5   0.0  0.5973  


In [6]:
ssaAvgTable = SolverSSA(model,'samples',2000,'seed',1,'verbose',True).getAvgTable()

SSA samples:    2000
SolverSSA analysis (method: default, lang: java) completed. Runtime: 0.451000 seconds.
      Station     JobClass    QLen    Util       RespT  ResidT  ArvR    Tput
0      Client  ClientClass  0.0000  0.0000  1.0000e-08  0.0000   0.0  3.1646
4  CacheDelay     HitClass  0.5675  0.5675  2.0000e-01  0.1646   0.0  2.8374
5  CacheDelay    MissClass  0.4325  0.4325  1.0000e+00  0.1768   0.0  0.4325


In [7]:
# ssaAvgTablePara = SolverSSA(model,'samples',20000,'seed',1,'verbose',True,'parallel').getAvgTable()